In [ ]:
# Step 0: Import Libraries
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama
from openai import OpenAI
import requests

# Step 1: Set Constants and Variables
print("[INFO] Setting constants and variable ...")
WEBSITE_URL = "https://arxiv.org/"
MODEL = "llama3.2"
approaches = ["local-call", "python-package", "openai-python-library"]
approach = approaches[2]

# Step 1: Scrape Website
print("[INFO] Scraping website ...")
url_response = requests.get(
    url=WEBSITE_URL,
    headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"}
    )
soup = BeautifulSoup(
    markup=url_response.content,
    features="html.parser"
    )
website_title = soup.title.string if soup.title else "No title found!!!"
for irrelevant in soup.body(["script", "style", "img", "input"]):
    irrelevant.decompose()
website_text = soup.body.get_text(
    separator="\n",
    strip=True
    )

# Step 2: Create Prompts
print("[INFO] Creating system prompt ...")
system_prompt = "You are an assistant that analyzes the contents of a \
    website and provides a short summary, ignoring text that might be \
    navigation related. Respond in markdown."

print("[INFO] Creating user prompt ...")
user_prompt = f"You are looking at a website titled {website_title}"
user_prompt += "\nBased on the contents of the website, please provide \
    a short summary of this website in markdown. If the website \
    includes news or announcements, summarize them, too. The contents \
    of this website are as follows:\n\n"
user_prompt += website_text

# Step 3: Make Messages List
print("[INFO] Making messages list ...")
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

# Step 4: Call Model and Print Results
if approach == "local-call":
    response = requests.post(
        url="http://localhost:11434/api/chat",
        json={
            "model": MODEL,
            "messages": messages,
            "stream": False
            },
        headers={"Content-Type": "application/json"}
    )
    print("[INFO] Printing result ...")
    display(Markdown(response.json()["message"]["content"]))
elif approach == "python-package":
    response = ollama.chat(
        model=MODEL,
        messages=messages,
        stream=False
    )
    print("[INFO] Printing result ...")
    display(Markdown(response["message"]["content"]))
elif approach == "openai-python-library":
    ollama_via_openai = OpenAI(
        base_url="http://localhost:11434/v1",
        api_key="ollama"
    )
    response = ollama_via_openai.chat.completions.create(
        model=MODEL,
        messages=messages
    )
    print("[INFO] Printing result ...")
    display(Markdown(response.choices[0].message.content))
else:
    raise ValueError(f"[INFO] Invalid approach! Please select an approach from {approaches} and try again.")